# Negamax with Alpha-Beta Pruning and Iterative Deepening

Joseph Griffith

Investigating the advantages of alpha-beta pruning applied to Tic-Tac-Toe.

# Classes

## Tic Tac Toe 

In [69]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9   #multiplies only the elements into a list
        self.player = 'X'
        if False:
            #self.board = ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']
            #startBoard = ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']
            #startBoard = ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']
            #startBoard = ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player
        self.movesExplored = 0

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')

        #0|1|2
        #3|4|5
        #6|7|8
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],   #rows
                [0, 3, 6], [1, 4, 7], [2, 5, 8],   #columns
                [0, 4, 8], [2, 4, 6]]              #diagonals
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored += 1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    
    def __str__(self):
        #s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)     #for a roomier board
        s = '\033[4m{}|{}|{}\n{}|{}|{}\n\033[0m{}|{}|{}\n'.format(*self.board)     #for a more compact board
        return s

    
    def getWinningValue(self):
        return 1
    
    def getBestMove(self, move1, move2):
        return move1 if (move1>move2) else move2
        
    def getNumberMovesExplored(self):
        return self.movesExplored

# Functions - 

## negamax()

In [70]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)   #if value == none, continue...
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

## opponent()

In [71]:
def opponent(board):
    return board.index(' ')   #just puts an O on the next blank

## playGame()

In [72]:
def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():   #check if O won
        #score, move = negamax(game,depthLimit)   #controls IDS numbers for NM
        score, move = negamaxIDS(game,depthLimit)   #controls IDS numbers for NM
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():   #check if X won
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            #print('Player', game.player, 'to', move)    #printing out the wrong player's move
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

## ebf()

In [73]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 0:
        return 0

    def ebfRec(low, high):
        mid = (low + high) * 0.5
        if mid == 1:
            estimate = 1 + depth
        else:
            estimate = (1 - mid**(depth + 1)) / (1 - mid)
        if abs(estimate - nNodes) < precision:
            return mid
        if estimate > nNodes:
            return ebfRec(low, mid)
        else:
            return ebfRec(mid, high)

    return ebfRec(1, nNodes)

## negamaxIDS()

In [74]:
def negamaxIDS(game, depthLimit):
    for i in range(1, depthLimit+1):
        #print('~~~~~~~~~~~~~~~~~ Game ' + str(i) + ' Start! ~~~~~~~~~~~~~~~~~')
        bestValue, bestMove = negamax(game, i)
#print('win: ', game.getWinningValue(), ', isover: ', game.isOver(), ', score: ', bestValue, ', move: ', bestMove)
        if game.getWinningValue() == bestValue:
            return bestValue, bestMove
    return bestValue, bestMove

In [75]:
#print(game.getMoves())
game = TTT()
playGame(game, opponent, 10)

 | | 
 | | 
 | | 

Player X to 0 for score 1
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 6 for score 1
X|O|O
X| | 
X| | 



In [76]:
game.isOver()

True

In [77]:
print('Number of moves explored', game.getNumberMovesExplored())

Number of moves explored 23338


## negamaxIDSab()

In [78]:
def negamaxIDSab(game, depthLimit):
    alpha = float('-inf')
    beta = float('inf')
    for i in range(1, depthLimit+1):
        bestValue, bestMove = negamaxab(game, i, alpha, beta)
        if game.getWinningValue() == bestValue:
            return bestValue, bestMove
    return bestValue, bestMove

## negamaxab()

In [79]:
def negamaxab(game, depthLeft, alpha, beta):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamaxab(game, depthLeft-1, -beta, -alpha)   #if value == none, continue...
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        if bestValue >= beta:
            return bestValue, bestMove
        alpha = bestValue if bestValue > alpha else alpha
    return bestValue, bestMove

## gamePlayer()

In [80]:
def gamePlayer(game,opponent,depthLimit, nmF):
    print(game)
    while not game.isOver():   #check if O won
        score, move = nmF(game,depthLimit)   #controls IDS numbers for NM
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():   #check if X won
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            #print('Player', game.player, 'to', move)    #printing out the wrong player's move
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()

## playGames()

In [81]:
def playGames(opponent, depthLimit):
    algs = ['negamax', 'negamaxIDS', 'negamaxIDSab']
    strings = []
    for i in range(3):
        game = TTT()
        print('~~~~~~~~~~~~~~~~~ ' + algs[i] + ' ~~~~~~~~~~~~~~~~~')    
        if i == 0:
            gamePlayer(game, opponent, depthLimit, negamax) 
        elif i == 1:
            gamePlayer(game, opponent, depthLimit, negamaxIDS) 
        elif i == 2: 
            gamePlayer(game, opponent, depthLimit, negamaxIDSab) 
        xmoves = game.board.count('X')
        moves = game.getNumberMovesExplored()
        depth = xmoves + game.board.count('O')
        #print('negamax made {} moves.\t {} moves explored for ebf({}, {}) of {}'.format(xmoves, moves, moves, depth, ebf(moves, depth)))
        strings.append('negamax made {} moves.\t {} moves explored for an\tebf({}, {}) of \t{}'.format(xmoves, moves, moves, depth, ebf(moves, depth)))
    #print(list(strings))
    for i in range(3):
        print(strings[i])

In [82]:
game = TTT()
playGames(opponent, 10)

~~~~~~~~~~~~~~~~~ negamax ~~~~~~~~~~~~~~~~~
 | | 
 | | 
 | | 

Player X to 0 for score 0
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 4 for score 1
X|O|O
X|X| 
 | | 

Player O to 5
X|O|O
X|X|O
 | | 

Player X to 6 for score 1
X|O|O
X|X|O
X| | 

~~~~~~~~~~~~~~~~~ negamaxIDS ~~~~~~~~~~~~~~~~~
 | | 
 | | 
 | | 

Player X to 0 for score 1
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 6 for score 1
X|O|O
X| | 
X| | 

~~~~~~~~~~~~~~~~~ negamaxIDSab ~~~~~~~~~~~~~~~~~
 | | 
 | | 
 | | 

Player X to 0 for score 1
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 6 for score 1
X|O|O
X| | 
X| | 

negamax made 4 moves.	 558334 moves explored for an	ebf(558334, 7) of 	6.46486057422959
negamax made 3 moves.	 23338 moves

* With vanilla negamax, the solution is not optimal because X could have won on it's 3rd turn, but chose a non-winning move instead. IDS and IDSab picked optimal paths to win.
* Each successive algorithm significantly reduces the number of nodes that it needs to explore, leading to huge decreases in run times.
* The EBF also decreases, in this instance, with the exception of IDS. Having saved two moves over negamax, IDS's EBF actually went up.

In [83]:
from time import time

In [84]:
def timeNM(opponent, depthLimit, nmF):
    start = time()
    game = TTT()
    gamePlayer(game, opponent, depthLimit, nmF)
    difference = time()-start
    print('time taken: ', difference)

In [85]:
timeNM(opponent, 10, negamax)

 | | 
 | | 
 | | 

Player X to 0 for score 0
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 4 for score 1
X|O|O
X|X| 
 | | 

Player O to 5
X|O|O
X|X|O
 | | 

Player X to 6 for score 1
X|O|O
X|X|O
X| | 

time taken:  15.784443140029907


In [86]:
timeNM(opponent, 10, negamaxIDS)

 | | 
 | | 
 | | 

Player X to 0 for score 1
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 6 for score 1
X|O|O
X| | 
X| | 

time taken:  0.7526190280914307


In [87]:
timeNM(opponent, 10, negamaxIDSab)

 | | 
 | | 
 | | 

Player X to 0 for score 1
X| | 
 | | 
 | | 

Player O to 1
X|O| 
 | | 
 | | 

Player X to 3 for score 1
X|O| 
X| | 
 | | 

Player O to 2
X|O|O
X| | 
 | | 

Player X to 6 for score 1
X|O|O
X| | 
X| | 

time taken:  0.23753118515014648


# Grading

In [88]:
%run -i grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
 | | 
 | | 

Player X to 0 for score 0
X| | 
 | | 
 | | 

Player O to 8
X| | 
 | | 
 | |O

Player X to 2 for score 1
X| |X
 | | 
 | |O

Player O to 7
X| |X
 | | 
 |O|O

Player X to 1 for score 1
X|X|X
 | | 
 |O|O


--- 10/10 points. playGame 